In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
data_benign = pd.read_csv('/content/drive/MyDrive/Co_che_ma_doc/datasets_DetectingMalwareUsingAHybridApproach-1/staDynBenignLab.csv')
data_mal1 = pd.read_csv('/content/drive/MyDrive/Co_che_ma_doc/datasets_DetectingMalwareUsingAHybridApproach-1/staDynVxHeaven2698Lab.csv')
data_mal2 = pd.read_csv('/content/drive/MyDrive/Co_che_ma_doc/datasets_DetectingMalwareUsingAHybridApproach-1/staDynVt2955Lab.csv')

In [ ]:
benign_filenames = data_benign['filename'].values  # lấy file name
mal2_filenames = data_mal2['filename'].values

# Load the grayscale images and labels
image_filenames = [] # tạo mảng chứa
labels = []

# Benign (staDynBenignLab.csv) 
for i in range(len(data_benign['filename'])):
  file_name = benign_filenames[i]
  image_filenames.append(f'/content/drive/MyDrive/Co_che_ma_doc/Benign-Images/{file_name}.png') # duyệt qua hết ảnh = file name
  labels.append(data_benign['label'][i]) # gắn nhãn vào từng ảnh

# Malware 1 (staDynVxHeaven2698Lab.csv)
for i in range(len(data_mal1['label'])):
  image_filenames.append(f'/content/drive/MyDrive/Co_che_ma_doc/Grayscale-Image/Malware-Images/staDynVxHeaven2698Lab/{i}_VxHeaven_malware.png')
  labels.append(data_mal1['label'][i])

# Malware 2 (staDynVt2955Lab.csv)
for i in range(len(data_benign['filename'])):
  file_name = mal2_filenames[i]
  image_filenames.append(f'/content/drive/MyDrive/Co_che_ma_doc/Malware-Images/staDynVt2955Lab/{file_name}.png')
  labels.append(data_mal2['label'][i])

In [ ]:
# Load and preprocess the images
images = []
for filename in image_filenames:
    img = Image.open(filename).convert('L') # chuyển ảnh về L
    img_array = np.array(img)
    images.append(img_array)

images = np.array(images)
labels = np.array(labels)

In [ ]:
# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [ ]:
# Normalize the image data
x_train = x_train / 255.0
x_test = x_test / 255.0
 # chuyển đổi dữ liệu
# Reshape the data for CNN input
x_train = x_train.reshape(-1, 33, 33, 1)
x_test = x_test.reshape(-1, 33, 33, 1)

# Convert labels to one-hot encoding
num_classes = 2
y_train = np.eye(num_classes)[y_train]
y_test = np.eye(num_classes)[y_test]

In [ ]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(33, 33, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
98/98 [==============================] - 6s 57ms/step - loss: 0.3532 - accuracy: 0.8540 - val_loss: 0.3637 - val_accuracy: 0.8239
Epoch 2/10
98/98 [==============================] - 8s 80ms/step - loss: 0.3327 - accuracy: 0.8582 - val_loss: 0.3400 - val_accuracy: 0.8458
Epoch 3/10
98/98 [==============================] - 4s 42ms/step - loss: 0.3075 - accuracy: 0.8637 - val_loss: 0.3289 - val_accuracy: 0.8522
Epoch 4/10
98/98 [==============================] - 6s 57ms/step - loss: 0.3051 - accuracy: 0.8740 - val_loss: 0.3444 - val_accuracy: 0.8290
Epoch 5/10
98/98 [==============================] - 4s 40ms/step - loss: 0.2973 - accuracy: 0.8717 - val_loss: 0.3218 - val_accuracy: 0.8522
Epoch 6/10
98/98 [==============================] - 4s 39ms/step - loss: 0.2950 - accuracy: 0.8695 - val_loss: 0.3474 - val_accuracy: 0.8303
Epoch 7/10
98/98 [==============================] - 6s 62ms/step - loss: 0.2915 - accuracy: 0.8736 - val_loss: 0.3272 - val_accuracy: 0.8470
Epoch 8/10
98

In [ ]:
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
y_pred = model.predict(x_test)
y_pred = np.argmax(y_pred, axis=1)

# Convert predictions to categorical
y_pred = to_categorical(y_pred)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

# Print the metrics
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-Score: ", f1)

25/25 [==============================] - 0s 10ms/step
Accuracy:  0.8534704370179949
Precision:  0.8392536581987059
Recall:  0.8534704370179949
F1-Score:  0.8278623427440452
